In [ ]:
%%capture
!pip install llama-index==0.10.37 cohere==5.5.0 openai==1.30.1 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 

previously had this (llama-index-llms-cohere) above but removing solved pydantic problem

# Load Variables and Models 

In [ ]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

sys.path.append('../helpers')

from utils import setup_llm

In [ ]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter CO API KEY: ")

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [ ]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [ ]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [ ]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model, setup_vector_store

COLLECTION_NAME = "rules-of-the-gaa"

setup_llm(
    provider="cohere",
    model="command-r-plus",
    api_key=CO_API_KEY
)

# removed model_name from parameters here, defined in utils
setup_embed_model(
    provider="openai",
    api_key=OPENAI_API_KEY
)

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

In [ ]:
Settings.llm

In [ ]:
Settings.embed_model

# Load Docstore

In [ ]:
from utils import get_documents_from_docstore

documents = get_documents_from_docstore("../data/rules-of-the-gaa")

In [ ]:
documents[0].__dict__

# Ingest to VectorDatabase


In [ ]:
from llama_index.core.constants import DEFAULT_CHUNK_SIZE
from llama_index.core.node_parser.text import SentenceSplitter
from llama_index.core import StorageContext
from utils import ingest

print(f"This is the chunk size: {DEFAULT_CHUNK_SIZE}")

tranforms = [
    SentenceSplitter(chunk_size=DEFAULT_CHUNK_SIZE),
    Settings.embed_model
]

nodes = ingest(
    documents=documents,
    transformations=tranforms,
    vector_store=vector_store,
)

# Build Index over Vector database

In [ ]:
from utils import create_index, create_query_engine

index = create_index(
    from_where="vector_store",
    embed_model=Settings.embed_model,
    vector_store=vector_store,
)

query_engine = create_query_engine(
    index=index,
    mode="query",
    # llm=Settings.llm
)

# Query

In [ ]:
from utils import create_query_pipeline

from llama_index.core.query_pipeline import InputComponent

input_component = InputComponent()

chain = [input_component, query_engine]

query_pipeline = create_query_pipeline(chain)

In [ ]:
response_1 = query_pipeline.run(input="How many points is a goal worth?")

In [ ]:
print(response_1)

In [ ]:
response_1.source_nodes[0].__dict__

In [ ]:
response_2 = query_pipeline.run(input="Is a goal worth 3 points?")

In [ ]:
print(response_2)

In [ ]:
response_2.source_nodes[0].__dict__

In [ ]:
len(response_2.source_nodes)

In [ ]:
response_3 = query_pipeline.run(input="In ladies football how many point is a 45 worth?")
print(response_3)

In [ ]:
response_3.source_nodes[0].__dict__